In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, balanced_accuracy_score

In [3]:
# -----------------------------
# Data Targets
# -----------------------------
# read file
data_path = '/Users/jonathankipping/code/syeda-tabassum-rahaman/scam-job-detector/raw_data/data_cleaned.csv'
df = pd.read_csv(data_path)
X = df.drop(columns=["fraudulent"])
y = df["fraudulent"]
# -----------------------------
# Combine text columns
# -----------------------------
text_columns = [
    "title",
    "company_profile",
    "description",
    "requirements",
    "benefits"
]
combined_text = X[text_columns].fillna("").agg(" ".join, axis=1).to_list()
# -----------------------------
# Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    combined_text, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
# -----------------------------
# Tokenizer
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
train_tokens = tokenizer(
    X_train,
    padding="max_length",
    truncation=True,
    max_length=256,
    return_tensors="np",
)
test_tokens = tokenizer(
    X_test,
    padding="max_length",
    truncation=True,
    max_length=256,
    return_tensors="np",
)
# -----------------------------
# Load BERT Tiny model
# -----------------------------
model = TFAutoModelForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny",
    num_labels=2,
    from_pt=True
)
# -----------------------------
# Compile Model
# -----------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)
# -----------------------------
# Fit Model
# -----------------------------
history = model.fit(
    {
        "input_ids": train_tokens["input_ids"],
        "attention_mask": train_tokens["attention_mask"],
    },
    y_train.values,
    validation_split=0.1,
    epochs=10,
    batch_size=16
)
# -----------------------------
# Evaluate
# -----------------------------
test_loss, test_acc = model.evaluate(
    {
        "input_ids": test_tokens["input_ids"],
        "attention_mask": test_tokens["attention_mask"],
    },
    y_test.values
)
print("Test Accuracy:", test_acc)

# Predict on test set
pred_logits = model.predict(
    {
        "input_ids": test_tokens["input_ids"],
        "attention_mask": test_tokens["attention_mask"],
    }
).logits

# Convert logits → class labels
y_prob = pred_logits[:,1]

y_prob
pd.DataFrame(y_prob).to_csv('../raw_data/bert_probas.csv')

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should p

Epoch 1/10
805/805 [==============================] - 105s 126ms/step - loss: 0.2023 - accuracy: 0.9527 - val_loss: 0.1273 - val_accuracy: 0.9644
Epoch 2/10
805/805 [==============================] - 95s 117ms/step - loss: 0.0953 - accuracy: 0.9706 - val_loss: 0.0627 - val_accuracy: 0.9818
Epoch 3/10
805/805 [==============================] - 94s 117ms/step - loss: 0.0523 - accuracy: 0.9840 - val_loss: 0.0584 - val_accuracy: 0.9769
Epoch 4/10
805/805 [==============================] - 93s 115ms/step - loss: 0.0361 - accuracy: 0.9897 - val_loss: 0.0613 - val_accuracy: 0.9797
Epoch 5/10
805/805 [==============================] - 94s 116ms/step - loss: 0.0299 - accuracy: 0.9911 - val_loss: 0.0578 - val_accuracy: 0.9811
Epoch 6/10
805/805 [==============================] - 94s 117ms/step - loss: 0.0203 - accuracy: 0.9944 - val_loss: 0.0642 - val_accuracy: 0.9832
Epoch 7/10
805/805 [==============================] - 94s 116ms/step - loss: 0.0154 - accuracy: 0.9959 - val_loss: 0.0584 - val_a

In [4]:
pd.DataFrame(y_prob).to_csv('../raw_data/bert_probas.csv')

In [5]:
# Predict on test set
pred = model.predict(
    {
        "input_ids": test_tokens["input_ids"],
        "attention_mask": test_tokens["attention_mask"],
    }
)
pred

112/112 [==============================] - 9s 77ms/step


TFSequenceClassifierOutput(loss=None, logits=array([[ 3.6307786, -3.7005801],
       [ 3.925928 , -3.991476 ],
       [ 3.881991 , -3.9434168],
       ...,
       [ 3.920732 , -3.9818761],
       [ 3.913963 , -3.9754841],
       [ 3.9302251, -3.9888659]], shape=(3576, 2), dtype=float32), hidden_states=None, attentions=None)